# Introduction

This notebook demostrates the use of PAM to undertake facility sampling.

> Facility sampling allows us to convert activity locations from a zone- to point-basis, compatible with agent-based models' requirements. It allows us to stochastically distribute demand across appropriate candidate locations (typically extracted from OSM), following a set of rules such as land use and supply or cost weights.

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pam

# Data

Load a dummy population to PAM:

In [ ]:
people_expanded=pd.read_csv('./intermediate_outputs/people_expanded.csv').set_index('pid')
hhs_expanded=pd.read_csv('./intermediate_outputs/hhs_expanded.csv').set_index('hid')
trips=pd.read_csv('./intermediate_outputs/trips_intermediate.csv').drop(['Unnamed: 0'],axis=1)

In [ ]:
population = read.load_travel_diary(
    trip_diary=trips,
    person_attributes=people_expanded,
    hh_attributes=hhs_expanded
)

# Facility Sampling